In [2]:
import os
import shutil
import zipfile
import json
import pandas as pd
from glob import glob

label_dir = r"C:\Users\USER\Desktop\동아리\O.P.C\프로젝트\라벨링_데이터_var\112.공적말하기_실습_및_평가_데이터\3.개방데이터\1.데이터\Validation\02.라벨링데이터"
unzip_dir = "./unzipped_라벨링_var"

part_files = sorted(glob(os.path.join(label_dir, "*.zip.part*")))

if not part_files:
    raise FileNotFoundError("병합할 .zip.part* 파일이 없습니다.")

zip_filename = part_files[0].split(".zip.part")[0] + ".zip"
merged_path = os.path.join(label_dir, zip_filename)

with open(merged_path, "wb") as outfile:
    for part in part_files:
        with open(part, "rb") as pf:
            shutil.copyfileobj(pf, outfile)

print(f"병합 완료: {merged_path}")

병합 완료: C:\Users\USER\Desktop\동아리\O.P.C\프로젝트\라벨링_데이터_var\112.공적말하기_실습_및_평가_데이터\3.개방데이터\1.데이터\Validation\02.라벨링데이터\VL_1.언어적_4._A03_30대.zip


In [3]:
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall("unzipped_data")

In [2]:
label_root = r"C:\Users\USER\Desktop\동아리\O.P.C\프로젝트\unzipped_data_var"
json_paths = glob(os.path.join(label_root, "**", "*.json"), recursive=True)

#빈 리스트 생성
label_dts = []

#json 파일 하나씩 열기
for path in json_paths:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    label_dt = {}

    # 발표자
    label_dt["gender"] = data.get("speaker", {}).get("gender", "")
    
    # 발표 스크립트 원문
    label_dt["presen_script"] = data.get("presentation", {}).get("presen_script", "")

    # 발표 내용
    label_dt["start_time"] = data.get("script", {}).get("start_time", "")
    label_dt["end_time"] = data.get("script", {}).get("end_time", "")
    label_dt["script_stt_txt"] = data.get("script", {}).get("script_stt_txt", "")
    label_dt["script_tag_txt"] = data.get("script", {}).get("script_tag_txt", "")
    label_dt["word_cnt"] = data.get("script", {}).get("word_cnt", 0)
    label_dt["audible_word_cnt"] = data.get("script", {}).get("audible_word_cnt", 0)
    label_dt["sent_cnt"] = data.get("script", {}).get("sent_cnt", 0)

    # 평가 점수(평균)
    label_dt["repeat_scr"] = data.get("average", {}).get("repeat_scr", 0.0)
    label_dt["filler_words_scr"] = data.get("average", {}).get("filler_words_scr", 0.0)
    label_dt["pause_scr"] = data.get("average", {}).get("pause_scr", 0.0)
    label_dt["wrong_scr"] = data.get("average", {}).get("wrong_scr", 0.0)
    label_dt["voc_quality_scr"] = data.get("average", {}).get("voc_quality_scr", 0.0)
    label_dt["voc_speed_sec_scr"] = data.get("average", {}).get("voc_speed_sec_scr", 0.0)
    label_dt["eval_grade"] = data.get("average", {}).get("eval_grade", "")

    # eval_grade 점수 변환
    label_dt["eval_grade"] = {"A+":8.0,"A0":7.0,"B+":6.0,"B0":5.0,"C+":4.0,"C0":3.0,"D+":2.0,"D0":1.0,"F":0.0}.get(label_dt["eval_grade"], None)

    # 파일명
    label_dt["filename"] = data.get("info", {}).get("filename", "")

    # 읽어들인 딕셔너리를 빈 리스트에 추가
    label_dts.append(label_dt)

# DataFrame으로 변환
df_presentations = pd.DataFrame(label_dts)

In [15]:
df_presentations.head()

,gender,presen_script,start_time,end_time,script_stt_txt,script_tag_txt,word_cnt,audible_word_cnt,sent_cnt,repeat_scr,filler_words_scr,pause_scr,wrong_scr,voc_quality_scr,vor_speed_sec_scr,eval_grade,filename
0,여자,"안녕하십니까? 여러분, 오늘은 e스포츠가 젊은이들에게 미치는 긍정적인 영향에 대해 ...",00:00,03:39,안녕하십니까 여러분. 오늘은 e스포츠가 젊은이들에게 미치는 긍정적인 영향에 대해서 ...,안녕하십니까 여러분. 오늘은 e스포츠가 젊은이들에게 미치는 긍정적인 영향에 대해서 ...,308,303,26,4.90,4.85,4.85,3.95,3.95,3.70,7.0,A03_S01_F_A_01_002_02_WA_MO_presentation.json
1,여자,"안녕하세요, 여러분. 오늘은 저의 제주도 가족 여행에 대한 추억을 나누고자 합니다....",00:00,05:23,안녕하세요 여러분 오늘은 저의 제주도 가족 여행에 대한 추억을 나누고자 합니다. 이...,안녕하세요 여러분 오늘은 저의 제주도 가족 여행에 대한 추억을 나누고자 합니다. 이...,369,335,30,1.05,4.85,4.80,2.90,2.45,1.05,3.0,A03_S01_F_E_02_076_02_WA_MO_presentation.json
2,여자,안녕하십니까? 오늘은 제가 얼마전에 다녀온 남해 여행에 대한 추억을 나누고자 합니다...,00:00,04:12,안녕하십니까? 오늘은 제가 얼마 전에 다녀온 남해 여행에 대한 추억을 나누고자 합니...,안녕하십니까? 오늘은 제가 얼마 전에 다녀온 남해 여행에 대한 추억을 나누고자 합니...,351,334,30,4.00,4.80,4.85,2.95,3.35,1.05,4.0,A03_S01_F_E_05_086_02_WA_MO_presentation.json
3,여자,"존경하는 여러분, 오늘은 인종차별이 이루어지면 안되는 이유에 대해 고찰하고자 합니다...",00:00,04:11,"존경하는 여러분, 오늘은 인종차별이 이루어지면 안되는 이유에 대해 오 고찰하고자 합...","존경하는 여러분, 오늘은 인종차별이 이루어지면 안되는 이유에 대해 <REP>오 고...",355,336,27,4.90,4.90,4.80,3.95,3.30,1.20,5.0,A03_S02_F_B_05_033_02_WA_MO_presentation.json
4,남자,"안녕하십니까, 오늘은 영화나 드라마 등 문화 콘텐츠에서 이루어지는 인종차별에 대한 ...",00:00,04:24,"안녕하십니까 오늘은 영화나 드라마나, 영화 오늘은 영화나 드라마 등 문화 콘텐츠에서...","안녕하십니까 <REP>오늘은 영화나 드라마나, 영화 오늘은 영화나 드라마</REP...",390,385,24,1.35,4.90,4.85,3.90,3.20,2.90,4.0,A03_S02_M_B_06_034_02_WA_MO_presentation.json


In [3]:
df_presentations.to_csv('extract_val_label_30.csv')

In [18]:
video_part_dir = r"C:\Users\USER\Desktop\동아리\O.P.C\프로젝트\원본_데이터_var\112.공적말하기_실습_및_평가_데이터\3.개방데이터\1.데이터\Validation\01.원천데이터"
unzip_dir = "./unzipped_영상"

part_files = sorted(glob(os.path.join(video_part_dir, "*.zip.part*")), key=lambda x: int(x.split("part")[-1]))

zip_filename = part_files[0].split(".zip.part")[0] + ".zip"
merged_zip_path = os.path.join(video_part_dir, zip_filename)

with open(merged_zip_path, "wb") as outfile:
    for part in part_files:
        with open(part, "rb") as pf:
            shutil.copyfileobj(pf, outfile)

print(f"병합 완료: {merged_zip_path}")

os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(merged_zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(f"압축 해제 완료: {unzip_dir}")

병합 완료: C:\Users\USER\Desktop\동아리\O.P.C\프로젝트\원본_데이터_var\112.공적말하기_실습_및_평가_데이터\3.개방데이터\1.데이터\Validation\01.원천데이터\VS_1.언어적_4._A03_30대.zip
압축 해제 완료: ./unzipped_영상


In [19]:
# 라벨링 데이터 - 원본 데이터 매핑
# 저장된 영상 파일 이름에 맞게 변환
df_presentations["file_stem"] = df_presentations["filename"].apply(
    lambda x: x.replace("_presentation.json", "")
)

# 영상 파일 이름 추출
video_dir = r"C:\Users\USER\Desktop\동아리\O.P.C\프로젝트\unzipped_영상_var"
video_files = {
    os.path.splitext(f)[0]: f
    for f in os.listdir(video_dir)
    if f.lower().endswith(".mp4")
}

# 존재 여부 매핑
df_presentations["video_exists"] = df_presentations["file_stem"].apply(
    lambda stem: stem in video_files
)

# 매칭된 발표만 필터링
df_matched = df_presentations[df_presentations["video_exists"] == True].copy()

print(f"매칭된 발표 수: {len(df_matched)} / {len(df_presentations)}")
df_matched.head()

매칭된 발표 수: 10 / 10


,gender,presen_script,start_time,end_time,script_stt_txt,script_tag_txt,word_cnt,audible_word_cnt,sent_cnt,repeat_scr,filler_words_scr,pause_scr,wrong_scr,voc_quality_scr,vor_speed_sec_scr,eval_grade,filename,file_stem,video_exists
0,여자,"안녕하십니까? 여러분, 오늘은 e스포츠가 젊은이들에게 미치는 긍정적인 영향에 대해 ...",00:00,03:39,안녕하십니까 여러분. 오늘은 e스포츠가 젊은이들에게 미치는 긍정적인 영향에 대해서 ...,안녕하십니까 여러분. 오늘은 e스포츠가 젊은이들에게 미치는 긍정적인 영향에 대해서 ...,308,303,26,4.90,4.85,4.85,3.95,3.95,3.70,7.0,A03_S01_F_A_01_002_02_WA_MO_presentation.json,A03_S01_F_A_01_002_02_WA_MO,True
1,여자,"안녕하세요, 여러분. 오늘은 저의 제주도 가족 여행에 대한 추억을 나누고자 합니다....",00:00,05:23,안녕하세요 여러분 오늘은 저의 제주도 가족 여행에 대한 추억을 나누고자 합니다. 이...,안녕하세요 여러분 오늘은 저의 제주도 가족 여행에 대한 추억을 나누고자 합니다. 이...,369,335,30,1.05,4.85,4.80,2.90,2.45,1.05,3.0,A03_S01_F_E_02_076_02_WA_MO_presentation.json,A03_S01_F_E_02_076_02_WA_MO,True
2,여자,안녕하십니까? 오늘은 제가 얼마전에 다녀온 남해 여행에 대한 추억을 나누고자 합니다...,00:00,04:12,안녕하십니까? 오늘은 제가 얼마 전에 다녀온 남해 여행에 대한 추억을 나누고자 합니...,안녕하십니까? 오늘은 제가 얼마 전에 다녀온 남해 여행에 대한 추억을 나누고자 합니...,351,334,30,4.00,4.80,4.85,2.95,3.35,1.05,4.0,A03_S01_F_E_05_086_02_WA_MO_presentation.json,A03_S01_F_E_05_086_02_WA_MO,True
3,여자,"존경하는 여러분, 오늘은 인종차별이 이루어지면 안되는 이유에 대해 고찰하고자 합니다...",00:00,04:11,"존경하는 여러분, 오늘은 인종차별이 이루어지면 안되는 이유에 대해 오 고찰하고자 합...","존경하는 여러분, 오늘은 인종차별이 이루어지면 안되는 이유에 대해 <REP>오 고...",355,336,27,4.90,4.90,4.80,3.95,3.30,1.20,5.0,A03_S02_F_B_05_033_02_WA_MO_presentation.json,A03_S02_F_B_05_033_02_WA_MO,True
4,남자,"안녕하십니까, 오늘은 영화나 드라마 등 문화 콘텐츠에서 이루어지는 인종차별에 대한 ...",00:00,04:24,"안녕하십니까 오늘은 영화나 드라마나, 영화 오늘은 영화나 드라마 등 문화 콘텐츠에서...","안녕하십니까 <REP>오늘은 영화나 드라마나, 영화 오늘은 영화나 드라마</REP...",390,385,24,1.35,4.90,4.85,3.90,3.20,2.90,4.0,A03_S02_M_B_06_034_02_WA_MO_presentation.json,A03_S02_M_B_06_034_02_WA_MO,True


In [20]:
import sys
!{sys.executable} -m pip install opencv-python

In [22]:
# 영상 재생 여부 확인
import cv2 # OpenCV 라이브러리 (영상 파일을 열고 프레임을 읽을 수 있음)

video_dir = r"C:\Users\USER\Desktop\동아리\O.P.C\프로젝트\unzipped_영상_var"
video_paths = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith(".mp4")]

results = []

for path in video_paths:
    filename = os.path.basename(path)
    cap = cv2.VideoCapture(path)

    if not cap.isOpened():
        results.append((filename, "열기 실패"))
        continue

    ret, frame = cap.read()
    if not ret:
        results.append((filename, "프레임 읽기 실패"))
    else:
        results.append((filename, "정상"))

    cap.release()

print(f"{len(results)}개 영상 점검 완료\n")
for fname, status in results:
    print(f"{status} - {fname}")

10개 영상 점검 완료

정상 - A03_S01_F_A_01_002_02_WA_MO.mp4
정상 - A03_S01_F_E_02_076_02_WA_MO.mp4
정상 - A03_S01_F_E_05_086_02_WA_MO.mp4
정상 - A03_S02_F_B_05_033_02_WA_MO.mp4
정상 - A03_S02_M_B_06_034_02_WA_MO.mp4
정상 - A03_S02_M_D_01_057_02_WA_MO.mp4
정상 - A03_S02_M_D_05_067_02_WA_MO.mp4
정상 - A03_S02_M_D_05_068_02_WA_MO.mp4
정상 - A03_S02_M_D_05_069_02_WA_MO.mp4
정상 - A03_S02_M_D_06_071_02_WA_MO.mp4


In [23]:
!pip install moviepy

In [24]:
from moviepy import VideoFileClip

In [25]:
# 비디오 - 오디오 변환
video_dir = r"C:\Users\USER\Desktop\동아리\O.P.C\프로젝트\unzipped_영상_var"
output_dir = "./wav"
os.makedirs(output_dir, exist_ok=True)

for idx, row in df_matched.iterrows():
    stem = row["file_stem"]
    mp4_path = os.path.join(video_dir, f"{stem}.mp4")
    wav_path = os.path.join(output_dir, f"{stem}.wav")
    
    try:
        video = VideoFileClip(mp4_path)
        video.audio.write_audiofile(wav_path, codec='pcm_s16le')  # wav 포맷
        print(f"변환 완료: {stem}.wav")
    except Exception as e:
        print(f"변환 실패: {stem} - {e}")

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 2883, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 218.23, 'bitrate': 3018, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], 'video_bitrate': 2883, 'video_fps': 30.0, 'default_audi

MoviePy - Done.
변환 완료: A03_S01_F_A_01_002_02_WA_MO.wav
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 2713, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 323.04, 'bitrate': 2848, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], 

MoviePy - Done.
변환 완료: A03_S01_F_E_02_076_02_WA_MO.wav
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 2699, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 251.4, 'bitrate': 2835, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], '

MoviePy - Done.
변환 완료: A03_S01_F_E_05_086_02_WA_MO.wav
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 10001, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 129, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 250.71, 'bitrate': 10137, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080]

MoviePy - Done.
변환 완료: A03_S02_F_B_05_033_02_WA_MO.wav
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 3029, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 263.4, 'bitrate': 3164, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], '

...

MoviePy - Done.
변환 완료: A03_S02_M_D_01_057_02_WA_MO.wav
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 2728, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 126, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 289.7, 'bitrate': 2861, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], '

MoviePy - Done.
변환 완료: A03_S02_M_D_05_067_02_WA_MO.wav
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 4658, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 125, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 264.13, 'bitrate': 4791, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], 

MoviePy - Done.
변환 완료: A03_S02_M_D_05_068_02_WA_MO.wav
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 3589, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 232.03, 'bitrate': 3724, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], 

MoviePy - Done.
변환 완료: A03_S02_M_D_05_069_02_WA_MO.wav
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.29.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 2742, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 288.6, 'bitrate': 2877, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], '

MoviePy - Done.
변환 완료: A03_S02_M_D_06_071_02_WA_MO.wav
